In [14]:
import datetime
import argparse
import random
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Parameter Setting

In [15]:
water_pressure_file = './simulate_pressure.csv'
water_head_file = './simulate_head.csv'

Adj_file = './AdjacencyMatrix781.csv'
node_heights = './node_heights.csv'
node_types = './node_types.csv'

model_file = 'GGNN_33_step12_height_type.pkl'
log_file = 'log_step12_height_type'

In [16]:
parser = argparse.ArgumentParser(description='AGN_Pressure_LTown_step12 Parameters')
parser.add_argument('--num_step'     , type=int,   default=105120,help='Total number of steps')
parser.add_argument('--num_hp'       , type=int,   default=12,  help='history steps equal to predict steps')
parser.add_argument('--num_vertex'   , type=int,   default=781, help='number of nodes')
parser.add_argument('--hidden_dim'   , type=int,   default=12,  help='hidden_dim')

parser.add_argument('--d'            , type=int,   default=781, help='number of nodes')
parser.add_argument('--bn_decay'     , type=float, default=0.1, help='batch normalization decay')
parser.add_argument('--iter_step'    , type=int,   default=3 ,  help='iter_step')
parser.add_argument('--train_ratio'  , type=float, default=0.7, help='train set ratio')
parser.add_argument('--val_ratio'    , type=float, default=0.2, help='validation set ratio')
parser.add_argument('--test_ratio'   , type=float, default=0.1, help='test set ratio')
parser.add_argument('--l1_lambda'    , type=float, default=0.01,help='L1 norm')
parser.add_argument('--batch_size'   , type=int,   default=32,  help='batch size')
parser.add_argument('--max_epoch'    , type=int,   default=10,  help='max epoch')
parser.add_argument('--patience'     , type=int,   default=10,  help='early stop patience')
parser.add_argument('--gamma'        , type=float, default=0.8, help='learning rate decay rate')
parser.add_argument('--learning_rate', type=float, default=1e-3,help='learning rate') 
parser.add_argument('--decay_epoch'  , type=int,   default=10,  help='decay epoch')
args = parser.parse_args(args=[])
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Utils

In [17]:
# Read command line arguments
def log_string(log, string):
    log.write(string + '\n')
    log.flush() 
    print(string)

# statistic model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [18]:
def apply_mask(args, num_masked, labels):
    ''' 
    Generate a two-step mask: 
    a one-step masking tensor for masking only sensors, 
    a two-step masking tensor for continuing to mask sensors

    1. sensor only: masked_tensor1 / masking method: mask1
    2. mask the sensor again: masked_tensor2 / masking method: mask2
    '''
    # '111' '226' '300' is the PRV outlet --> fixed pressure
    fixed_nodes = ['111', '226', '300', '781']
    sensor_name = ['1','4','31','54','105','114','163','188','215','229',
    '288','296','330','340','408','413','427','456','467',
    '493','504','514','517','547','611','634','642','677',
    '720','724','738','750','767']

    sensor_name.extend(fixed_nodes)

    # Reduce the selection probability of areas with few sensors 
    # 1:3:29 --> 87:29:1
    weights = [     29 if i in ['1', '4', '31'] 
               else 1 if i == '215' 
               else 87 for i in sensor_name]

    # Make sure fixed_sensors have a weight of 0 to indicate that they will not be masked
    for sensor in fixed_nodes:
        weights[sensor_name.index(sensor)] = 0

    # Converts sensor_index to an integer
    sensor_index = [int(i)-1 for i in sensor_name]
 
    # Create a mask with the same shape as the input tensor
    mask1 = torch.zeros(labels.shape)
    mask1[:,:,sensor_index] = 1
    
    # The index is randomly selected in sensor_index to mask
    masked_indices = random.choices(sensor_index, weights=weights, k=num_masked)
    
    # Create a second mask
    mask2 = mask1.clone()
    mask2[:, :, masked_indices] = 0

    mask1 = mask1.to(args.device)
    mask2 = mask2.to(args.device)
    
    # Apply the mask to the input tensor
    masked_tensor1 = labels * mask1
    masked_tensor2 = labels * mask2
    masked_tensor1 = torch.tensor(masked_tensor1, dtype=torch.float32)
    masked_tensor2 = torch.tensor(masked_tensor2, dtype=torch.float32)

    return masked_tensor1, masked_tensor2, mask1, mask2

In [19]:
# Divide the time series data into multiple training samples
class CustomDataset(Dataset):
    def __init__(self, args, water_pressure_file, water_head_file, dataset_type='train'):
        self.args = args
        self.water_head_file = water_head_file
        self.dataset_type = dataset_type
        
        # Calculate the number of time steps for the training set, verification set, and test set
        train_steps = round(args.train_ratio * args.num_step)
        test_steps = round(args.test_ratio * args.num_step)
        val_steps = args.num_step - train_steps - test_steps

        fixed_nodes = ['111', '226', '300', '781']
        sensor_name = ['1','4','31','54','105','114','163','188','215','229',
                       '288','296','330','340','408','413','427','456','467',
                       '493','504','514','517','547','611','634','642','677',
                       '720','724','738','750','767']
        sensor_name.extend(fixed_nodes)
        sensor_index = [int(i)-1 for i in sensor_name]
        
        # Read water head and pressure data 
        pressure = pd.read_csv(water_pressure_file, header=0, index_col=None)
        pressure = torch.from_numpy(pressure.values)
        std_pressure = pressure[sensor_index].std()     # Avoid information leaks
        mean_pressure  = pressure[sensor_index].mean()  # Avoid information leaks
        water_pressure =  (pressure - mean_pressure) / std_pressure 

        head = pd.read_csv(water_head_file, header=0, index_col=None)
        head = torch.from_numpy(head.values)
        std_head = head[sensor_index].std()   # Avoid information leaks
        mean_head = head[sensor_index].mean() # Avoid information leaks
        water_head = (head - mean_head) / std_head

        # Split the data set
        self.train_head = water_head[: train_steps]
        self.train_pressure = water_pressure[: train_steps]
        self.val_head = water_head[train_steps: train_steps + val_steps]
        self.val_pressure = water_pressure[train_steps: train_steps + val_steps]
        self.test_head = water_head[-test_steps:]
        self.test_pressure = water_pressure[-test_steps:]

        # Divide the time series data into multiple training samples
        self.trainX, self.trainY = self.seq2instance(args, self.train_head, self.train_pressure)
        self.valX, self.valY = self.seq2instance(args, self.val_head, self.val_pressure)
        self.testX, self.testY = self.seq2instance(args, self.test_head, self.test_pressure)
    
    # Slide window to generate sample
    def seq2instance(self, args, data1, data2):
        num_step, dims = data1.shape
        num_sample = num_step - args.num_hp - args.num_hp + 1
        x = torch.zeros(num_sample, args.num_hp, dims)
        y = torch.zeros(num_sample, args.num_hp, dims)
        for i in range(num_sample):
            ''' 
            head = pressure + height
            
            Although this paper uses two kinds of data to improve the meaning of the data
            But (x[i] --> data1) or (y[i] --> data2)
            You can get basically the same results
            But the effect will be slightly lower in the range of 0.5m by about 2%
            
            '''
            x[i] = data2[i: i + args.num_hp]
            y[i] = data1[i + args.num_hp: i + args.num_hp + args.num_hp]
        return x, y

    def __len__(self):
        if self.dataset_type == 'train':
            return len(self.trainX)
        elif self.dataset_type == 'val':
            return len(self.valX)
        elif self.dataset_type == 'test':
            return len(self.testX)
    
    def __getitem__(self, idx):
        if self.dataset_type == 'train':
            return self.trainX[idx].to(self.args.device), self.trainY[idx].to(self.args.device)
        elif self.dataset_type == 'val':
            return self.valX[idx].to(self.args.device), self.valY[idx].to(self.args.device)
        elif self.dataset_type == 'test':
            return self.testX[idx].to(self.args.device), self.testY[idx].to(self.args.device)

In [20]:
# Custom loss function
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
    def forward(self, pred, target):
        mask = target != 0   
        loss = (pred - target) ** 2
        loss = loss * mask   
        loss = torch.sum(loss)/torch.sum(mask)
        return loss

In [21]:
# initialize the weights of the model
''' 
xavier_normal initialization is better than kaiming_normal initialization
'''
def init_weights(m):
    if isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0)

# Model

In [22]:
# Entities should not be multiplied unnecessarily.

class GGNN(nn.Module):
    def __init__(self, num_hp, num_vertex, hidden_dim, batch_size, K, d, iter_step, bn_decay):
        super(GGNN, self).__init__()
        self.num_hp = num_hp            
        self.num_vertex = num_vertex    
        self.hidden_dim = hidden_dim     
        self.batch_size = batch_size
        self.iter_step = iter_step    

        self.W_z = AttLayer(num_vertex, hidden_dim, batch_size, d)   # update gate
        self.W_r = AttLayer(num_vertex, hidden_dim, batch_size, d)   # reset gate
        self.W   = AttLayer(num_vertex, hidden_dim, batch_size, d)   # candidate hidden state

        self.W_height = AttLayer(num_vertex, hidden_dim, batch_size, d)
        self.W_type   = AttLayer(num_vertex, hidden_dim, batch_size, d)

        self.FC_height = nn.Linear(num_hp + num_hp, num_vertex)  
        self.FC_type   = nn.Linear(num_hp + num_hp, num_vertex)

        self.FC1 = nn.Linear(num_hp + num_vertex, num_vertex)
        self.FC2 = nn.Linear(num_hp + num_vertex, num_vertex)

        self.output_layer = nn.Linear(num_vertex, num_hp)

        self.BatchNorm = nn.BatchNorm1d(num_vertex)

        self.tanh = nn.Tanh()
        
    def forward(self, X, Adj, node_height, node_type):
        """
        Adj : adjacency matrix (num_nodes, num_nodes)
        X   : input features   (batch_size, num_hp, num_nodes)
        node height ：  add
        node type   :   multiply
        """
        h = torch.zeros(self.num_vertex, self.num_vertex).to(X.device) 
        Adj = Adj.unsqueeze(0).expand(X.shape[0], -1, -1)  

        for _ in range(self.iter_step):
            ''' 
            Do not arbitrarily increase the number of loops, it will run very slowly.
            ''' 

            a = torch.matmul(Adj, h)  # Aggregate neighbor information to the hidden layer

            # hidden state update
            concat = torch.cat((X, a), dim=1) 
            concat = concat.permute(0, 2, 1)  
            concat = self.FC1(concat)  
            concat = self.BatchNorm(concat)

            attn_z = self.W_z(concat, concat, concat) # reset gate: self-Atten
            attn_r = self.W_r(concat, concat, concat) # update gate: self-Atten
            attn_z = torch.sigmoid(attn_z)  # This parameter can be replaced by the constant 0.5
            attn_r = torch.sigmoid(attn_r)  # This parameter can be replaced by the constant 0.5

            concat_r = torch.cat((X, attn_r * a), dim=1) 
            concat_r = concat_r.permute(0, 2, 1)
            concat_r = self.FC2(concat_r)  

            h_tilde = self.W(concat_r, concat_r, concat_r)  # candidate hidden state: self-Atten
            h_tilde = torch.tanh(h_tilde)

            h = (1 - attn_z) * a + attn_z * h_tilde  # gate control
            
        # Secondary polymerization (elevation)
        concat_height = torch.cat((X, node_height), dim=1) 
        concat_height = concat_height.permute(0, 2, 1)  
        concat_height = self.FC_height(concat_height)  
        concat_height = self.W_height(concat_height, h, concat_height) # Atten from h
        concat_height = self.BatchNorm(concat_height)
        # Secondary polymerization (type)
        concat_type = torch.cat((X, node_type), dim=1) 
        concat_type = concat_type.permute(0, 2, 1)  
        concat_type = self.FC_type(concat_type)  
        concat_type = self.W_type(concat_type, h, concat_type) # Atten from h
        concat_type = self.BatchNorm(concat_type)
        
        # feature fusion
        h = torch.add(h, concat_height) 
        h = torch.matmul(h, concat_type) 

        output = self.output_layer(h)    
        output = output.permute(0, 2, 1)
        output = self.tanh(output) 

        return output

class AttLayer(nn.Module):
    def __init__(self, num_vertex, hidden_dim, batch_size, d):
        super(AttLayer, self).__init__()

        self.d = d
        self.batch_size = batch_size

        self.FC_query = nn.Linear(num_vertex, hidden_dim)
        self.FC_key   = nn.Linear(num_vertex, hidden_dim)
        self.FC_value = nn.Linear(num_vertex, hidden_dim)

        self.FC_ = nn.Linear(hidden_dim, num_vertex)  

        # It is very important to prevent overfitting
        # adjust this parameter if the effect is not good
        self.Dropout = nn.Dropout(0.8)  

    def forward(self, input1, input2, input3):
        ''' 
        this model is simplified from the multi-headed attention mechanism
        multi-headed attention mechanism has no advantage in this case
        '''

        query = self.FC_query(input1)
        key   = self.FC_key(input2)
        value = self.FC_value(input3)

        attention = torch.matmul(query, key.transpose(1, 2))
        attention /= (self.d ** 0.5) 
        attention = F.softmax(attention, dim=-1)

        X = torch.matmul(attention, value)
        X = self.Dropout(X)

        output = self.FC_(X) 

        del query, key, value, attention, X
        return output

# Train

In [23]:
def train(args, std_pressure, mean_pressure, model, model_file, log, loss_criterion, optimizer, scheduler):

    train_dataset = CustomDataset(args, water_pressure_file, water_head_file, dataset_type='train')
    val_dataset = CustomDataset(args, water_pressure_file, water_head_file, dataset_type='val')
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    wait = 0
    val_loss_min = float('inf')
    best_model_wts = None
    train_total_loss = []
    val_total_loss = []

    for epoch in range(args.max_epoch):
        if wait >= args.patience:
            log_string(log, f'early stop at epoch: {epoch:04d}')
        i = 0

        start_train = time.time()
        model.train()
        train_loss = 0
        for batch in train_dataloader:
            X, Y = batch

            # Check and fill dimensions
            target_size = args.batch_size
            pad_size = target_size - X.size(0)
            if pad_size > 0:
                X = F.pad(X, (0, 0, 0, 0, pad_size, 0))
                Y = F.pad(Y, (0, 0, 0, 0, pad_size, 0))

            # Cascade masking 
            # (1) Only 33 sensor data; (2): Continue mask at 33 sensor positions)
            num_masked = 33 - epoch * 3 + 1   
            _, X2, _, mask2 = apply_mask(args, num_masked, X)
            Y2 = Y * mask2
            node_heights_input = node_heights * mask2
            node_types_input = node_types * mask2

            optimizer.zero_grad()
            
            pred = model(X2, Adj, node_heights_input, node_types_input)
            loss_batch = loss_criterion(pred, Y2)

            # L1 norm (L2 norm is not suitable for this model)
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            loss_batch += args.l1_lambda * l1_norm

            train_loss += float(loss_batch) * args.batch_size
            loss_batch.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            i += 1
            if i % 10 == 0:
                log_string(log, f'Training batch: {i+1} in epoch:{epoch+1}, training batch loss:{loss_batch:.4f}')
                
            del X2, mask2, Y2, loss_batch, node_heights_input, node_types_input

        train_loss = train_loss * std_pressure / (105120 * args.train_ratio)

        train_total_loss.append(train_loss)
        end_train = time.time()

        # val loss
        j = 0
        start_val = time.time()
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for batch in val_dataloader:
                X, Y= batch

                # Check and fill dimensions
                target_size = args.batch_size
                pad_size = target_size - X.size(0)
                if pad_size > 0:
                    X = F.pad(X, (0, 0, 0, 0, pad_size, 0))
                    Y = F.pad(Y, (0, 0, 0, 0, pad_size, 0))

                ## Cascade masking
                num_masked = 33 - epoch * 3 + 1
                X1, _, mask1, _ = apply_mask(args, num_masked, X)
                Y1 = Y * mask1
                node_heights_input = node_heights * mask1
                node_types_input = node_types * mask1
                pred = model(X1, Adj, node_heights_input, node_types_input)
                
                # Losses are calculated only for nodes with masks
                loss_batch = loss_criterion(pred, Y1) 
                val_loss += loss_batch * args.batch_size
                j += 1

                del X1,  mask1, Y1, loss_batch, node_heights_input, node_types_input

        val_loss /= 105120 * args.val_ratio
        val_total_loss.append(val_loss)
        end_val = time.time()

        log_string(
            log,
            '%s | epoch: %04d/%d, training time: %.1fs, inference time: %.1fs' %
            (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), epoch + 1, args.max_epoch, end_train - start_train, end_val - start_val))
        log_string(
            log, f'train loss: {train_loss:.4f}, val_loss: {val_loss:.4f}')
        
        if val_loss <= val_loss_min:
            log_string(
                log,
                f'val loss decrease from {val_loss_min:.4f} to {val_loss:.4f}, saving model to {model_file}')
            wait = 0
            val_loss_min = val_loss
            best_model_wts = model.state_dict()
        else:
            wait += 1
        scheduler.step()

    model.load_state_dict(best_model_wts)
    torch.save(model, model_file)
    log_string(log, f'Training and validation are completed, and model has been stored as {model_file}')
    
    return train_total_loss, val_total_loss

# Test

In [24]:
def test(args, std_head, mean_head, model_file, log):
    log_string(log, '**** testing model ****')
    log_string(log, 'loading model from %s' % model_file)

    model = torch.load(model_file)
    model.eval()

    test_dataset = CustomDataset(args, water_pressure_file, water_head_file, dataset_type='test')
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # test model
    model = torch.load(model_file)
    log_string(log, 'model restored!')
    log_string(log, 'evaluating...')

    Pred_1 = torch.Tensor().to(args.device)
    Pred_2 = torch.Tensor().to(args.device)
    Pred_3 = torch.Tensor().to(args.device)
    Pred_4 = torch.Tensor().to(args.device)
    Pred_5 = torch.Tensor().to(args.device)
    Pred_6 = torch.Tensor().to(args.device)
    Pred_7 = torch.Tensor().to(args.device)
    Pred_8 = torch.Tensor().to(args.device)
    Pred_9 = torch.Tensor().to(args.device)
    Pred_10 = torch.Tensor().to(args.device)
    Pred_11 = torch.Tensor().to(args.device)
    Pred_12 = torch.Tensor().to(args.device)

    Label_1 = torch.Tensor().to(args.device)
    Label_2 = torch.Tensor().to(args.device)
    Label_3 = torch.Tensor().to(args.device)
    Label_4 = torch.Tensor().to(args.device)
    Label_5 = torch.Tensor().to(args.device)
    Label_6 = torch.Tensor().to(args.device)
    Label_7 = torch.Tensor().to(args.device)
    Label_8 = torch.Tensor().to(args.device)
    Label_9 = torch.Tensor().to(args.device)
    Label_10 = torch.Tensor().to(args.device)
    Label_11 = torch.Tensor().to(args.device)
    Label_12 = torch.Tensor().to(args.device)

    mae_list = []
    mape_list = []

    with torch.no_grad():
        start_test = time.time()
        for batch in test_dataloader:
            X, Y = batch

            target_size = args.batch_size
            pad_size = target_size - X.size(0)
            if pad_size > 0:
                X = F.pad(X, (0, 0, 0, 0, pad_size, 0))
                Y = F.pad(Y, (0, 0, 0, 0, pad_size, 0))

            num_masked = 0
            X1, _, _, _,  = apply_mask(args, num_masked, X)  
            # Only X1 is available to avoid information leakage
            pred = model(X1, Adj, node_heights, node_types) 
            loss_batch = pred * std_head + mean_head
            Y = Y * std_head + mean_head
            
            # MAE/MAPE
            mae  = torch.mean(torch.abs(loss_batch - Y))
            mape = torch.mean(torch.abs(loss_batch - Y) / Y)*100
            mae_list.append(mae.item())
            mape_list.append(mape.item())

            loss_batch_ = torch.mean(loss_batch, dim=0)
            testPred_1  = loss_batch_[0]
            testPred_2  = loss_batch_[1]
            testPred_3  = loss_batch_[2]
            testPred_4  = loss_batch_[3]
            testPred_5  = loss_batch_[4]
            testPred_6  = loss_batch_[5]
            testPred_7  = loss_batch_[6]
            testPred_8  = loss_batch_[7]
            testPred_9  = loss_batch_[8]
            testPred_10 = loss_batch_[9]
            testPred_11 = loss_batch_[10]
            testPred_12 = loss_batch_[11]
            testPred_1 = testPred_1.unsqueeze(0)
            testPred_2 = testPred_2.unsqueeze(0)
            testPred_3 = testPred_3.unsqueeze(0)
            testPred_4 = testPred_4.unsqueeze(0)
            testPred_5 = testPred_5.unsqueeze(0)
            testPred_6 = testPred_6.unsqueeze(0)
            testPred_7 = testPred_7.unsqueeze(0)
            testPred_8 = testPred_8.unsqueeze(0)
            testPred_9 = testPred_9.unsqueeze(0)
            testPred_10 = testPred_10.unsqueeze(0)
            testPred_11 = testPred_11.unsqueeze(0)
            testPred_12 = testPred_12.unsqueeze(0)
            Pred_1 = torch.cat((Pred_1, testPred_1), dim=0)
            Pred_2 = torch.cat((Pred_2, testPred_2), dim=0)
            Pred_3 = torch.cat((Pred_3, testPred_3), dim=0)
            Pred_4 = torch.cat((Pred_4, testPred_4), dim=0)
            Pred_5 = torch.cat((Pred_5, testPred_5), dim=0) 
            Pred_6 = torch.cat((Pred_6, testPred_6), dim=0)
            Pred_7 = torch.cat((Pred_7, testPred_7), dim=0)
            Pred_8 = torch.cat((Pred_8, testPred_8), dim=0)
            Pred_9 = torch.cat((Pred_9, testPred_9), dim=0)
            Pred_10 = torch.cat((Pred_10, testPred_10), dim=0)
            Pred_11 = torch.cat((Pred_11, testPred_11), dim=0)
            Pred_12 = torch.cat((Pred_12, testPred_12), dim=0)
            
            Y_ = torch.mean(Y, dim=0)
            testLabel_1  = Y_[0] 
            testLabel_2  = Y_[1] 
            testLabel_3  = Y_[2] 
            testLabel_4  = Y_[3] 
            testLabel_5  = Y_[4] 
            testLabel_6  = Y_[5] 
            testLabel_7  = Y_[6] 
            testLabel_8  = Y_[7] 
            testLabel_9  = Y_[8] 
            testLabel_10 = Y_[9] 
            testLabel_11 = Y_[10]
            testLabel_12 = Y_[11] 
            testLabel_1 = testLabel_1.unsqueeze(0)
            testLabel_2 = testLabel_2.unsqueeze(0)
            testLabel_3 = testLabel_3.unsqueeze(0)
            testLabel_4 = testLabel_4.unsqueeze(0)
            testLabel_5 = testLabel_5.unsqueeze(0)
            testLabel_6 = testLabel_6.unsqueeze(0)
            testLabel_7 = testLabel_7.unsqueeze(0)
            testLabel_8 = testLabel_8.unsqueeze(0)
            testLabel_9 = testLabel_9.unsqueeze(0)
            testLabel_10 = testLabel_10.unsqueeze(0)
            testLabel_11 = testLabel_11.unsqueeze(0)
            testLabel_12 = testLabel_12.unsqueeze(0)
            Label_1 = torch.cat((Label_1, testLabel_1), dim=0)
            Label_2 = torch.cat((Label_2, testLabel_2), dim=0)
            Label_3 = torch.cat((Label_3, testLabel_3), dim=0)
            Label_4 = torch.cat((Label_4, testLabel_4), dim=0)
            Label_5 = torch.cat((Label_5, testLabel_5), dim=0)
            Label_6 = torch.cat((Label_6, testLabel_1), dim=0)
            Label_7 = torch.cat((Label_7, testLabel_2), dim=0)
            Label_8 = torch.cat((Label_8, testLabel_3), dim=0)
            Label_9 = torch.cat((Label_9, testLabel_4), dim=0)
            Label_10 = torch.cat((Label_10, testLabel_5), dim=0)
            Label_11 = torch.cat((Label_11, testLabel_6), dim=0)
            Label_12 = torch.cat((Label_12, testLabel_7), dim=0)
  
    end_test = time.time()

    log_string(log, 'testing time: %.1fs' % (end_test - start_test))

    return (Pred_1, Pred_2, Pred_3, Pred_4, Pred_5, Pred_6, Pred_7, Pred_8, Pred_9, Pred_10, Pred_11, Pred_12, 
            Label_1, Label_2, Label_3, Label_4, Label_5, Label_6, Label_7, Label_8, Label_9, Label_10, Label_11, Label_12, 
            mae_list, mape_list)

# Main

In [25]:
# Create a description file
log = open(log_file, 'w')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
log_string(log, f'using device: {device}')

log_string(log, '——loading data——')

fixed_nodes = ['111', '226', '300', '781']
sensor_name = ['1','4','31','54','105','114','163','188','215','229',
'288','296','330','340','408','413','427','456','467',
'493','504','514','517','547','611','634','642','677',
'720','724','738','750','767']
sensor_name.extend(fixed_nodes)
sensor_index = [int(i)-1 for i in sensor_name]

water_head = pd.read_csv(water_head_file, header=0, index_col=None)
water_head= torch.from_numpy(water_head.values)
std_head = water_head[sensor_index].std()
mean_head = water_head[sensor_index].mean()
std_head = std_head.to(device)
mean_head = mean_head.to(device)
log_string(log, f'mean head at sensor nodes: {mean_head}, std head at sensor nodes: {std_head}')
std_head_ = water_head.std()
mean_head_ = water_head.mean()
log_string(log, f'mean head : {mean_head_}, std pressure: {std_head_}')

# Adjacency Matrix
Adj = pd.read_csv(Adj_file, header=0, index_col=0, sep=',')
Adj = torch.from_numpy(Adj.values)
Adj = (Adj + torch.eye(Adj.shape[0])).to(dtype=torch.float32)
Adj = Adj.to(device)
log_string(log, f'Adjacency matrix loaded')

# node feature
node_heights = pd.read_csv(node_heights, header=None, index_col=0, sep=',')
node_heights = torch.from_numpy(node_heights.values).float()
node_heights = (node_heights - node_heights.mean()) / (node_heights.std())
node_heights = node_heights.unsqueeze(0).expand(args.batch_size, -1, -1)
node_heights = node_heights.repeat(1, 1, 12)
node_heights = node_heights.transpose(1, 2)
node_heights = node_heights.to(device)
log_string(log, f'node heights loaded')

node_types = pd.read_csv(node_types, header=None, index_col=0, sep=',')
node_types = torch.from_numpy(node_types.values).float()
node_types = (node_types - node_types.mean()) / (node_types.std())
node_types = node_types.unsqueeze(0).expand(args.batch_size, -1, -1)
node_types = node_types.repeat(1, 1, 12)
node_types = node_types.transpose(1, 2)
node_types = node_types.to(device)
log_string(log, f'node types loaded')

using device: cuda
——loading data——
mean head at sensor nodes: 73.0772963793718, std head at sensor nodes: 6.528225559049961
mean head : 72.97857368764194, std pressure: 6.515785156814622
Adjacency matrix loaded
node heights loaded
node types loaded


In [26]:
# load data
log_string(log, '——loading model——')
model = GGNN(args.num_hp, args.num_vertex, args.hidden_dim,  
             args.batch_size, args.K, args.d, args.iter_step, args.bn_decay).to(args.device)
model.apply(init_weights)
log_string(log, 'model loaded!')

loss_criterion = CustomLoss()
optimizer = optim.Adam(model.parameters(), args.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer,
                                      step_size=args.decay_epoch,
                                      gamma=args.gamma)
parameters = count_parameters(model)
log_string(log, 'trainable parameters: {:,}'.format(parameters))
log_string(log, torch.__version__)

——loading model——
model loaded!
trainable parameters: 1,478,292
2.3.0+cu121


In [27]:
if __name__ == '__main__':
    start = time.time()

    loss_train, loss_val = train(args, std_head, mean_head, model, model_file, log, loss_criterion, optimizer, scheduler)

    (testPred_1, testPred_2, testPred_3, testPred_4, testPred_5, testPred_6, testPred_7, testPred_8, testPred_9, testPred_10, testPred_11, testPred_12, 
    testLabel_1, testLabel_2, testLabel_3, testLabel_4, testLabel_5, testLabel_6, testLabel_7, testLabel_8, testLabel_9, testLabel_10, testLabel_11, testLabel_12, 
    mae_list, mape_list) = test(args, std_head, mean_head, model_file, log)

    end = time.time()
    log_string(log, 'total time: %.1fmin' % ((end - start) / 60))
    log.close()

C:\AppData\Local\Temp\ipykernel_34960\1302657811.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masked_tensor1 = torch.tensor(masked_tensor1, dtype=torch.float32)
C:\AppData\Local\Temp\ipykernel_34960\1302657811.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masked_tensor2 = torch.tensor(masked_tensor2, dtype=torch.float32)


Training batch: 11 in epoch:1, training batch loss:345.0157
Training batch: 21 in epoch:1, training batch loss:244.4195
Training batch: 31 in epoch:1, training batch loss:163.1194
Training batch: 41 in epoch:1, training batch loss:108.5228
Training batch: 51 in epoch:1, training batch loss:69.4231
Training batch: 61 in epoch:1, training batch loss:44.9714
Training batch: 71 in epoch:1, training batch loss:31.9676
Training batch: 81 in epoch:1, training batch loss:24.0109
Training batch: 91 in epoch:1, training batch loss:19.5300
Training batch: 101 in epoch:1, training batch loss:17.1739
Training batch: 111 in epoch:1, training batch loss:15.8323
Training batch: 121 in epoch:1, training batch loss:14.1766
Training batch: 131 in epoch:1, training batch loss:14.0706
Training batch: 141 in epoch:1, training batch loss:12.5691
Training batch: 151 in epoch:1, training batch loss:11.8852
Training batch: 161 in epoch:1, training batch loss:11.8917
Training batch: 171 in epoch:1, training batc

C:\AppData\Local\Temp\ipykernel_34960\1302657811.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masked_tensor1 = torch.tensor(masked_tensor1, dtype=torch.float32)
C:\AppData\Local\Temp\ipykernel_34960\1302657811.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  masked_tensor2 = torch.tensor(masked_tensor2, dtype=torch.float32)


testing time: 36.1s
total time: 124.7min


# Export data for analysis

In [30]:
# 12 forecasting steps
save_test_cpu = testPred_1.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("1_test_height_type.csv", index=False)

save_test_cpu = testPred_2.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("2_test_height_type.csv", index=False)

save_test_cpu = testPred_3.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("3_test_height_type.csv", index=False)

save_test_cpu = testPred_4.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("4_test_height_type.csv", index=False)

save_test_cpu = testPred_5.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("5_test_height_type.csv", index=False)

save_test_cpu = testPred_6.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("6_test_height_type.csv", index=False)

save_test_cpu = testPred_7.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("7_test_height_type.csv", index=False)

save_test_cpu = testPred_8.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("8_test_height_type.csv", index=False)

save_test_cpu = testPred_9.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("9_test_height_type.csv", index=False)

save_test_cpu = testPred_10.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("10_test_height_type.csv", index=False)

save_test_cpu = testPred_11.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("11_test_height_type.csv", index=False)

save_test_cpu = testPred_12.cpu() 
save_test_numpy = save_test_cpu.numpy()
save_test_DF = pd.DataFrame(save_test_numpy)
save_test_DF = save_test_DF.iloc[:-12, : ]
save_test_DF.to_csv("12_test_height_type.csv", index=False)

In [31]:
# label output
save_label_cpu = testLabel_1.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("1_label_height_type.csv", index=False)

save_label_cpu = testLabel_2.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("2_label_height_type.csv", index=False)

save_label_cpu = testLabel_3.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("3_label_height_type.csv", index=False)

save_label_cpu = testLabel_4.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("4_label_height_type.csv", index=False)

save_label_cpu = testLabel_5.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("5_label_height_type.csv", index=False)

save_label_cpu = testLabel_6.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("6_label_height_type.csv", index=False)

save_label_cpu = testLabel_7.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("7_label_height_type.csv", index=False)

save_label_cpu = testLabel_8.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("8_label_height_type.csv", index=False)

save_label_cpu = testLabel_9.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("9_label_height_type.csv", index=False)

save_label_cpu = testLabel_10.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("10_label_height_type.csv", index=False)

save_label_cpu = testLabel_11.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("11_label_height_type.csv", index=False)

save_label_cpu = testLabel_12.cpu() 
save_label_numpy = save_label_cpu.numpy()
save_label_DF = pd.DataFrame(save_label_numpy)
save_label_DF = save_label_DF.iloc[:-12, :]
save_label_DF.to_csv("12_label_height_type.csv", index=False)

In [32]:
# mae_list, mape_list
mae_list = pd.DataFrame(mae_list)
mae_list.to_csv("Time_mae_list_height_type.csv", index=False)

mape_list = pd.DataFrame(mape_list)
mape_list.to_csv("Time_mape_list_height_type.csv", index=False)